# Introduction to SQLAlchemy

- __0.__ Introduction
- __1.__ Importing necessary libraries
- __2.__ Creating engine
- __3.__ Creating the connection
- __4.__ Inspecting the database
- __5.__ Querying the database
- __6.__ Read a View to DataFrame
- __7.__ Example of Using a CTE
- __8.__ Create a New Table from DataFrame
- __9.__ Delete a table
- __10.__ Workflow example

## 0. Introduction
SQLAlchemy is a library that facilitates communication between Python programs and databases. 

There are two ways of working with SQLAlchemy:
- SQLAlchemy Core: the foundational architecture for SQLAlchemy as a “database toolkit”. It provides tools for managing connectivity to a database, interacting with database queries and results, and construction of SQL statements
- SQLAlchemy ORM:  builds upon the Core to provide optional object-relational mapping capabilities as it represents database relations as Python objects.

In this course, we will focus on the SQLAlchemy Core architecture, which gives us the following advantages:
- Learning the basic concepts in SQLAlchemy
- practicing the use of the Pandas library
- practicing the use of SQL statements

In [ ]:
!pip install sqlalchemy
!pip install pyodbc

## 1. Importing necessary libraries

In [ ]:
from sqlalchemy import create_engine, Table, inspect
import pandas as pd

## 2. Creating engine

In order to connect to a database, first we need to create an engine. The engine references are:
- a __Dialect__: There are several types of RDBMS, which we call dialects in SQLAlchemy. They all use SQL as a base, but they have slight differences. \
The most common RDBMS are: MSSQL, MySQL, PostgreSQL, Oracle, SQLite. \
Each dialect in SQLAlchemy has a standard DBAPI, which serves as a bridge between Python programs and the relational databases, and they standardize the way to perform database operations.
- a __Pool__: a pool will establish a connection at the specified server location
- a __Database__: the database name we want to connect to
- __username__ and __password__: when using SQL Server identication, we need to provide the username and password to be able to connect to the database. When using the Microsoft Authentication method, the username and password is not required, as the user is automatically identified by the Microsoft signin credentials. 

__NOTE__: by creating an engine, we did not yet connect to the database, we simply gave the instructions of how and where to connect.

In [ ]:
def new_engine(dialect, server, database, user=None, password=None, integrated_security=True):
    if integrated_security:
        # For Windows authentication
        eng = f"{dialect}://{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server"
    else:
        assert user, 'You must define a username'
        assert password, 'You must define a password'
        # For SQL Server authentication
        eng = f"{dialect}://{user}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
    print(eng)
    return create_engine(eng)

In [ ]:
# For Windows authentication
# Replace the server argument with the Server Name found when logging into SQL Server Management Studio 
# OR
# For SQL Server authentication
# Replace the server, user and password argument with the Server Name, username and password
engine = new_engine('mssql', 'DESKTOP-CIKOHBH', 'AdventureWorks2022')

In [ ]:
# alternative instead of the long code:
engine = create_engine("mssql://DESKTOP-CIKOHBH/AdventureWorks2022?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server")

In [ ]:
print(type(engine))

## 3. Creating the connection

The engine class instance we created has the .connect() method, which returns a Connection object and technically creates the connection between the database and our Python application.

In [ ]:
connection = engine.connect()

In [ ]:
print(type(connection))

## 4. Inspecting the database

Now that we have a connection to the database, we can inspect its contents. \
The AdventureWorks database has multiple schemas which we can list with the following command.

In [ ]:
inspector = inspect(engine)
schemas = inspector.get_schema_names()
print(schemas)

In [ ]:
for schema in schemas:
    print(schema)

Each schema in the database logically groups together tables, views, indexes and procedures. Schemas also help with security, as we can grant permission for users into specific schemas. \
In this example we list all the available tables in the Sales schema. All these tables are related to sales or the sales department.

In [ ]:
print(inspector.get_table_names(schema='Sales'))

In [ ]:
for col in inspector.get_columns(table_name='Customer', schema='Sales'):
#     print(col)
    print(col['name'])

## 5. Querying the database

The Pandas library can directly connect to and query a database with the .read_sql() method. The two most important arguments of the method are:
- sql: this is the SQL command that the query will execute  
- con: the connection we defined to the database

The result is a Pandas DataFrame.

In [ ]:
df = pd.read_sql(sql="SELECT * from Sales.Customer", con=connection)

In [ ]:
df.head(12)

The SQL query can get as complex as the user wants.\
Here is an example of multiple JOIN statements

In [ ]:
multijoin_sql ="""
SELECT
  c.CustomerID, 
  c.StoreID, 
  c.AccountNumber, 
  soh.SalesOrderID, 
  sod.ProductID
FROM Sales.Customer AS c 
INNER JOIN Sales.SalesOrderHeader AS soh 
    ON c.CustomerID = soh.CustomerID
INNER JOIN Sales.SalesOrderDetail AS sod 
    ON soh.SalesOrderID = sod.SalesOrderID
"""
join_df = pd.read_sql(sql=multijoin_sql, con=connection)

In [ ]:
join_df.head()

## 6. Read a View to DataFrame
Reading in Views into a DataFrame is the exact same process as reading the tables, we just need to specify which view to read

In [ ]:
view_sql = """
SELECT * 
FROM HumanResources.vEmployee
"""

view = pd.read_sql(sql=view_sql, con=connection)

In [ ]:
view.head()

## 7. Example of Using a CTE (Common Table Expression)
Common Table Expressions are a great tool for managing more complex queries.\
Basically we create intermediate temporary tables, which we use later in further queries.

In [ ]:
cte_sql = """
WITH CTE_Employee AS (
  SELECT 
    P.BusinessEntityID, 
    P.FirstName, 
    P.LastName, 
    HR.JobTitle 
  FROM HumanResources.Employee AS HR 
  INNER JOIN Person.Person AS P 
      ON HR.BusinessEntityID = P.BusinessEntityID 
  WHERE 
    Gender = 'M'
) 
SELECT 
  CTE_Employee.BusinessEntityID, 
  CTE_Employee.FirstName, 
  CTE_Employee.LastName, 
  CTE_Employee.JobTitle, 
  SUM(Sales.SubTotal) TotalSum 
FROM CTE_Employee 
INNER JOIN Sales.SalesOrderHeader AS Sales 
    ON CTE_Employee.BusinessEntityID = Sales.SalesPersonID 
GROUP BY 
  CTE_Employee.BusinessEntityID, 
  CTE_Employee.FirstName, 
  CTE_Employee.LastName, 
  CTE_Employee.JobTitle 
ORDER BY
  CTE_Employee.BusinessEntityID;

"""

result = pd.read_sql(cte_sql, engine)
result

## 8. Create a New Table from DataFrame
Creating new tables and adding them to our database is straightforward. \
We create/define our DataFrame and then we use the pandas.to_sql() method. \
We must define:
- the __name__ of the table
- the connection we are using (__con__)
- the __schema__ we want to create the table in (if not provided the table will be added the the default dbo schema) 
- if we want to have a separate index column or keep the columns as is (__index__)
- what SQLAlchemy should do if the table already exists (__if_exists__)

In [ ]:
table_data = {'Sketch':['Cheese shop sketch', 'Silly walks', 'Spanish inquisition', 'Lumberjack song', 'Argument clinic'],
        'Length':['5:29', '4:05', '8:17', '2:41', '6:22']
       }

df2 = pd.DataFrame(data=table_data)
df2

In [ ]:
df2.to_sql(name='MontyPython', con=engine, schema='Sales', index=False, if_exists='replace')

In [ ]:
# Save the table into the default dbo schema. dbo stands for DataBase Owner
df2.to_sql(name='MontyPython', con=engine, index=False, if_exists='replace')

## 9. Delete a table

In [ ]:
# Reading the table / Check existence
pd.read_sql("SELECT * FROM Sales.MontyPython", engine)

In [ ]:
# Executing the DROP TABLE command in Pandas
from pandas.io import sql
sql.execute('DROP TABLE IF EXISTS Sales.MontyPython', connection)

In [ ]:
# Reading the table / Check existence
pd.read_sql("SELECT * FROM Sales.MontyPython", engine)

## 10. Workflow example
In this part, we will go through an example of a workflow:
- Read: reading in data from the database into a Pandas DataFrame
- Update: modifying the data in the DataFrame
- Save: saving the modified DataFrame into a new database table

#### Reading in data

In [ ]:
# Read: reading in data from the database into a Pandas DataFrame
# Products ranked based on sold amount between 2011-07-01 and 2011-08-01
example_sql ="""
SELECT 
  DENSE_RANK() OVER (ORDER BY SUM(SOD.OrderQty) DESC) AS SalesRank,
  P.ProductID,
  P.Name,
  SUM(SOD.OrderQty) AS TotalSold
FROM 
  Production.Product AS P 
  JOIN Sales.SalesOrderDetail AS SOD ON P.ProductID = SOD.ProductID 
  JOIN Sales.SalesOrderHeader AS SOH ON SOD.SalesOrderID = SOH.SalesOrderID
WHERE 
  SOH.OrderDate BETWEEN '2011-07-01' AND '2011-07-31'
GROUP BY
  P.ProductID, P.Name
ORDER BY
  SUM(SOD.OrderQty) DESC
"""

example_df = pd.read_sql(example_sql, engine, index_col='SalesRank')
example_df

#### Modifying the DataFrame

In [ ]:
# Update: modifying the data in the DataFrame
# Adding a Goal column to th DataFrame, with the next month (August) sales goal of +10% TotalSold
example_df['August_Goal'] = round(example_df['TotalSold'] * 1.1, 0).astype(int)
example_df

#### Saving the modified DataFrame

In [ ]:
# Save: saving the modified DataFrame into a new database table
example_df.to_sql(name='August_Goal_Table', con=engine, schema='Sales', index=False, if_exists='replace')

#### Controlling the result

In [ ]:
# Double checking if the new table was created
check = pd.read_sql('SELECT * FROM Sales.August_Goal_Table', engine)
check